In [1]:
from ephysvibe.structures.neuron_data import NeuronData
import numpy as np
from sklearn.decomposition import PCA,SparsePCA
import matplotlib.pyplot as plt
import os
import glob
from ephysvibe.trials import align_trials
from ephysvibe.trials.spikes import firing_rate
import platform
from joblib import Parallel, delayed
from tqdm import tqdm
import h5py
from pathlib import Path
from typing import Dict,List
from ephysvibe.task import task_constants
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
seed=2024

In [2]:
def from_python_hdf5(load_path: Path):
    """Load data from a file in hdf5 format from Python."""
    with h5py.File(load_path, "r") as f:
        data = []
        for i_g in f.keys():
            group = f[i_g]
            dataset={}
            for key, value in zip(group.keys(), group.values()):
                dataset[key] = np.array(value)
            data.append(dataset)
    f.close()
    return data

In [3]:
if platform.system() == "Linux":
    basepath = (
        "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/new_structure/"
    )
elif platform.system() == "Windows":
    basepath = "C:/Users/camil/Documents/int/"

In [4]:
session = '2022-12-06_11-12-04'
areas=['pfc','v4']
subject='Riesling'
time_before=200

In [5]:
units_info = {'v4':[],'pfc':[]}
for area in areas:
    neu_path = basepath + "session_struct/" + area + "/neurons/*neu.h5"
    path_list = np.array(glob.glob(neu_path))
    mask=[]
    for path in path_list:
        mask.append(session == path.rsplit("/")[-1].rsplit("_" + subject)[0])
    path_list=path_list[mask]
    

In [6]:
neu_data = NeuronData.from_python_hdf5(path)

# get spike matrices in and out conditions
sp_in, mask_in = align_trials.get_align_tr(
    neu_data, select_block=1, select_pos=1, time_before=time_before,event="sample_on"
)

In [8]:
neu_data.code_samples

array([[1000., 1017., 1028., ...,   nan,   nan,   nan],
       [1000., 1009., 1022., ...,   nan,   nan,   nan],
       [1000., 1006., 1012., ...,   nan,   nan,   nan],
       ...,
       [1000., 1004., 1015., ..., 4746.,   nan,   nan],
       [1000., 1002., 1014., ...,   nan,   nan,   nan],
       [1000., 1004., 1015., ...,   nan,   nan,   nan]])

In [ ]:
np.concatenate(sp_in[:,neu_data.])

In [7]:
def get_unit(path):
    neu_data = NeuronData.from_python_hdf5(path)

    # get spike matrices in and out conditions
    sp_in, mask_in = align_trials.get_align_tr(
        neu_data, select_block=1, select_pos=1, time_before=time_before,event="sample_on"
    )
    for i_
    sp_in

SyntaxError: invalid syntax (857246847.py, line 8)